<a href="https://colab.research.google.com/github/lunduniversity/schoolprog-satellite/blob/master/exercises/gapminder/gapminder2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inspirationsuppgift: Plottar med Gapminder-data


I denna uppgift ska vi jämföra olika länders koldioxidutsläpp och några olika sätt man kan mäta det på. 

Vi kommer inte förklara detaljerna i programmeringen, men du kommer se vad programmering kan användas till.

Vi kommer att använda data från Gapminder - en svensk stiftelse med målet att göra statistik lättillgänglig och bidra till en god global utveckling. Gapminder grundades av bl.a. Hans Rosling. Du kan läsa mer om Gapminder på [Gapminders hemsida](https://www.gapminder.org) och på [wikipedia-artikeln om Gapminder](https://sv.wikipedia.org/wiki/Gapminder).


# 1. Köra kod i Google Colaboratory
Vi börjar med att kontrollera att vi kan köra programkod.

---

**Uppgift 1a:** Du behöver vara uppkopplad ("connected") till Colab för att kunna köra kod.

  * Uppe till höger i fönstret kan det stå "RAM" och "Disk". I så fall är du uppkopplad. I annat fall bör det finnas en knapp `Connect` (eller `Anslut`). Klicka i så fall på den.
  * Efter en stund bör du se att det står "RAM" och Disk" uppe till höger i stället för `Connect`-knappen. Då är du uppkopplad och kan gå vidare.

---

**Uppgift 1b:** Prova nu att köra "kodcellen" nedan genom att klicka på koden och sedan på den lilla pilen till vänster i kodcellen.

Första gången du kör något kan det komma upp en varning som du behöver godkänna.



In [ ]:
for _ in range(3):
  print('hej!')

Fick du det att fungera? Vad skrivs ut?

<details>
<summary markdown="span">
Svar
</summary>
Datorn skriver ut 'hej!' tre gånger 
</details>

---

**Uppdrag 1c:** Prova att ändra från 3 till 5 i programmet. Kör koden igen (genom att klicka på den lilla pilen till vänster i kodcellen). Vad händer? Prova gärna fler ändringar, t.ex. ändra texten `'hej!` till något annat.

---

# 2. Totala koldioxidutsläpp

Vi ska nu se på hur mycket koldioxid vi människor har släppt ut i hela världen fram till 2014. 

---
**Uppgift 2a**:
Kör först kodrutan med texten "Ladda ner datafiler" nedanför. Då körs kod som laddar ner data från Gapminder, och omvandlar den till variabler i Python.

In [1]:
#@title Ladda ner datafiler { display-mode: "form" }
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import math
url_pop = 'https://raw.githubusercontent.com/open-numbers/ddf--gapminder--population_historic/master/ddf--datapoints--population_total--by--geo--time.csv'
url_co2_ton_per_cap = 'https://raw.githubusercontent.com/open-numbers/ddf--gapminder--co2_emission/master/ddf--datapoints--co2_emissions_tonnes_per_person--by--country--year.csv'
url_income_ppp = 'https://raw.githubusercontent.com/open-numbers/ddf--gapminder--gdp_per_capita_cppp/master/ddf--datapoints--income_per_person_gdppercapita_ppp_inflation_adjusted--by--geo--time.csv'
url_country_co2_year = 'https://raw.githubusercontent.com/open-numbers/ddf--gapminder--co2_emission/master/ddf--datapoints--yearly_co2_emissions_1000_tonnes--by--country--year.csv'
url_kumu_co2 = 'https://raw.githubusercontent.com/open-numbers/ddf--gapminder--co2_emission/master/ddf--datapoints--cumulative_co2_emissions_tonnes--by--country--year.csv'
url_country_info = 'https://raw.githubusercontent.com/open-numbers/ddf--open_numbers/master/ddf--entities--geo--country.csv'

co2_per_cap = pd.read_csv(url_co2_ton_per_cap, error_bad_lines=False)
income_per_cap = pd.read_csv(url_income_ppp, error_bad_lines=False)
country_co2 = pd.read_csv(url_country_co2_year, error_bad_lines=False)
country_population = pd.read_csv(url_pop, error_bad_lines=False)
kumu_co2 = pd.read_csv(url_kumu_co2, error_bad_lines=False)
country_info = pd.read_csv(url_country_info, error_bad_lines=False)

country_info = country_info[['country', 'name', 'world_4region']]
income_per_cap = income_per_cap.rename(columns={'geo': 'country', 'time':'year', 'income_per_person_gdppercapita_ppp_inflation_adjusted': 'gdpPercap'})
income_per_cap = income_per_cap.merge(country_info, on='country')
co2_per_cap = co2_per_cap.rename(columns={'co2_emissions_tonnes_per_person': 'co2Percap'})
co2_per_cap = co2_per_cap.merge(country_info, on='country')
country_co2 = country_co2.merge(country_info, on='country')
country_co2 = country_co2.rename(columns={'yearly_co2_emissions_1000_tonnes': 'yearly_emissions_tonnes'})
country_co2['yearly_emissions_tonnes'] = country_co2["yearly_emissions_tonnes"].map(lambda x: round(x*1000))
kumu_co2 = kumu_co2.merge(country_info, on='country')
country_population = country_population.rename(columns={'time': 'year', 'geo': 'country', 'population_total': 'population'})

print("Klart!")

Klart!


---

Nu ska vi se hur mycket koldioxid vi människor har släppt ut genom åren.

---

**Uppgift 2b:** Kör koden nedan för att se utsläppen från 1800 till 2014. Prova sedan att ändra `from_year` och `to_year` för att "zooma in" på något annat intervall. Datan började samlas in 1800, så du kan inte se vad som vad som hände innan dess. Vad visas i grafen?

In [2]:

from_year = 1800 # Prova att ändra till något annat startår!
to_year = 2014   # Prova att ändra till något annat slutår! 

# välj ut år efter from_year och to_year
country_co2_from_year = country_co2[(from_year <= country_co2['year']) & (country_co2['year'] <= to_year)] 

# skapa och rita figuren
fig = px.bar(country_co2_from_year, x="year", y="yearly_emissions_tonnes",
             labels={"yearly_emissions_tonnes": "co2-utsläpp i ton",
                     "year": "år"})
fig.update_traces(hovertemplate=None, hoverinfo='skip')
fig.show()

Du kanske ser vita streck i ditt diagram. Det uppkommer p.g.a. sättet som vi ritar upp grafen på.

---

**Uppgift 2c:** Vi kan nu se hur koldioxidutsläppen har ändrat sig över åren. Hur har utvecklingen sett ut?

<details>
<summary markdown="span">
Svar
</summary>
Det har släppts ut mer och mer koldioxid för varje år.
</details>

---

**Uppgift 2d:** Om du väljer ett år tillräckligt långt bak i tiden kan du bland annat se att koldioxidutsläppen minskade kring 1990. Kan du komma på någon bra förklaring till detta?

<details>
<summary markdown="span">
Svar
</summary>
Sovietunionen upplöstes 1991 vilket på flera sätt ledde till lägre koldioxidutsläpp. Många industrier och jordbruk lades ner. Konsumtionen av kött minskade betydligt. Se t.ex. denna <a href="https://www.nature.com/articles/d41586-019-02024-6">artikel i Nature</a>.
</details>

---

**Uppgift 2e:** Enheten på y-axeln är ton, B står för miljarder (från engelskans billions). Kan du komma fram till hur ungefär hur mycket koldioxid vi släppte ut år 2014? Ändra intervallet så du kan läsa av värdet.


<details>
<summary markdown="span">
Svar
</summary>
År 2014 släppte vi ut strax under 35 miljarder ton koldioxid.
</details>

Hur mycket blir det per person? Räkna med att det ungefär fanns 7 miljarder människor år 2014.

<details>
<summary markdown="span">
Svar
</summary>
Vi släppte ut cirka 5 ton koldioxid per person under 2014
</details>

---


**Uppgift 2f:** Vilket land släppte ut mest ett specifikt år?Vi ska försöka få svar på detta genom att plotta cirkeldiagram för olika år. Sätt ett värde på variabeln `year` nedan. Förslagsvis kan du titta på ditt födelseår. Kör sedan kodrutan. Vad är det som visas?

In [ ]:
year = ??? # välj ett år!

# välj ut data från det valda året
country_co2_year = country_co2[country_co2['year'] == year]

total_co2 = sum(country_co2_year['yearly_emissions_tonnes'])

# rita ut cirkeldiagrammet
fig = go.Figure(data=[go.Pie(labels=country_co2_year['name'],
                             values=country_co2_year['yearly_emissions_tonnes'])])
fig.update_traces(hoverinfo='label+percent', textposition='inside', textinfo='percent')

fig.update_layout(
    title_text='Totalt utsläpp under ' + str(year) + ': ' + str(round(total_co2, -8)) + ' ton CO2'
)
fig.show()

---

**Uppgift 2g:** Vilket land släppte ut mest koldioxid för ditt år? Hovra över diagrammet för att se vilken tårtbit som tillhör vilket land. 

---

**Uppgift 2h:** Vi ska nu kolla lite närmare på 2014. Byt ut `year` till 2014 och kör kodrutan igen. Vi ser då hur det såg ut under 2014.

Vilket land släppte ut mest under 2014? Hur mycket släppte de ut? Använd gärna kodrutan nedan för att räkna ut mängden.

<details>
<summary markdown="span">
Svar
</summary>
Kina släppte ut mest, cirka 10 miljarder ton koldioxid.
</details>

---

In [ ]:
# Räkna ut mängden nedan
total_co2 = 12345000000   # Byt ut mot totala mängden ton co2
andel_i_procent = 17      # Byt ut mot ett procenttal
ton_co2 = total_co2 * andel_i_procent / 100
mton_co2 = ton_co2/1000000000
print("Utsläpp:", ton_co2, "ton")
print("Dvs", mton_co2, "miljarder ton")

# 3. Kumulativa utsläpp

Från cirkeldigrammet över utsläppen för år 2014 verkar det kanske som att Kina är den största utsläppsboven. Dock visar detta inte hela bilden. En koldioxidpartikel som släpptes ut 1920 är fortfarande kvar i atmosfären och bidrar till den globala uppvärmningen.

För att hitta vilket land som bidragit mest till den höga halten av koldioxid i atmosfären kan vi då istället titta på den totala mängden koldioxid varje land har släppt ut. Vi kallar detta för *kumulativa* koldioxidutsläpp. Som tidigare så visar vi utsläppen med ett cirkeldiagram. 

---

**Uppgift 3a:** kör kodrutan nedan. Vad är det som visas?


In [ ]:
# välj ut kumulativa utsläpp fram till 2014
kumu_co2 = kumu_co2[kumu_co2['year'] == 2014]


# rita ut cirkeldiagrammet
fig = go.Figure(data=[go.Pie(labels=kumu_co2['name'],
                             values=kumu_co2['cumulative_co2_emissions_tonnes'])])
fig.update_traces(hoverinfo='label+percent', textposition='inside', textinfo='percent')

fig.update_layout(
    title_text='Totala utsläpp fram tills 2014: ' + str(round(int(total_co2), -8)) + ' ton CO2'
)
fig.show()

---

**Uppgift 3b:** Vilket land har störst kumulativa utsläpp?

<details>
<summary markdown="span">
Svar
</summary>
Andelen ser annorlunda ut för de kumulativa utsläppen jämfört med utsläppen för 2014. Kina har fortfarande stora utsläpp men USA står för den största delen med ett utsläpp på 376 miljarder ton. 
</details>


---

**Uppgift 3c:** Vi ska nu titta lite närmare på Storbritannien. Hur stor andel av utsläppen står Storbritannien för i den kumulativa datan? Hur stor är andelen av koldioxidutsläppen för endast 2014?

<details>
<summary markdown="span">
Svar
</summary>
Storbritannien stod för 1,23% av utsläppen år 2014 men hela 5,27% i den kumulativa datan. 
</details>

Kommer du på någon förklaring till detta?

<details>
<summary markdown="span">
Svar
</summary>
En möjlig förklaring är att Storbritannien, Tyskland och andra västeuropeiska länder industraliserades tidigt och har därför "hunnit" släppa ut mer koldioxid än länder som Indien, vars industrialisering kom senare.
</details>

---

# 4. Utsläpp per capita

Nu ska vi titta på hur mycket utsläpp som varje land har per person. 

**Uppgift 4a:** Välj igen ett år som du vill titta på och kör kodrutan.

In [ ]:
# välj ett år!
year = 2014

# tar fram data för det valda året
co2_per_cap_year = co2_per_cap[co2_per_cap['year'] == year].copy()

# ritar digrammet
fig = px.bar(co2_per_cap_year, x="name", y="co2Percap",
             labels={"yearly_emissions_tonnes": "co2-utsläpp i tusentals ton",
                     "year": "år"},
             color="world_4region")
fig.show()

---

**Uppgift 4b:** Vad står färgerna för? 
<details>
<summary markdown="span">
Svar
</summary>
Vi delar upp världens länder i fyra delar: Afrika, Amerika (Syd- och nordamerika), Asien (med Australien) och Europa.
<br>
Varje färg står för en sådan del. 
</details>

---
**Uppgift 4c:** Testa några olika år. Kan du se några skillnader mot tidigare grafer? Vilka länder verkar släppa ut mest koldioxid per capita? Har de något gemensamt? 

<details>
<summary markdown="span">
Svar
</summary>
De länder med allra högst utsläpp per capita verkar i hög grad vara oljeproducerande länder.
</details>

---

Vi ska nu titta på hur de olika länderna står sig mot hela världens medelvärde. Vi gör detta genom att dela upp länderna i två kategorier: de som släpper ut mer än medelvärdet per person och de som släpper ut mindre per person. 

Sedan visar vi detta på en karta där de två kategorierna representeras av två olika färger.
 
--- 

**Uppgift 4d:** Fyll i medelutsläppet i ton per år. Ändra alltså värdet på variabeln `average_co2`. Vi räknade ut detta värde i uppgift 2e så scrolla upp dit om du inte kommer ihåg. 

Kör sedan kodrutan. Vad är det som visas?



In [ ]:
average_co2 = ??? # skriv in medelvärdet av utsläpp per capita

# formatterar till ISO-alpha-3
co2_per_cap_year.loc[: , 'country'] = co2_per_cap_year['country'].map(lambda s: s.upper())
co2_per_cap_year.loc[:, 'emission_category'] = co2_per_cap_year.apply(lambda row: 1 if row.co2Percap > average_co2 else 2, axis=1)
co2_per_cap_year

max_val = math.ceil(max(co2_per_cap_year['co2Percap']))
mid_val = co2_per_cap_year['co2Percap'].mean() / max_val
fig = px.choropleth(co2_per_cap_year, locations="country",
                    color="emission_category",
                    hover_name="co2Percap" # column to add to hover information
                    )
fig.update(layout_coloraxis_showscale=False)
fig.show()


---

**Uppgift 4f:** Ser du några mönster? Finns det något gemensamt med länder som, per capita,  släpper ut mindre än medelvärdet? Hur är det med länder som släpper ut mycket per capita?

---

Något man måste tänka på är att denna data endast visar territoriella utsläpp, alltså utsläppen som sker inom ett lands gränser. När vi i Sverige köper en t-shirt som har tillverkats i Bangladesh så räknas de utsläpp som krävdes för att tillverka tröjan till Bangladeshs siffror. 

Om man istället skulle titta på konsumtionsbaserade utsläpp så skulle Sverige ligga över medelvärdet av utsläpp per capita.

<hr size=5 noshade>

# 5. Avslutning

Vi har nu tittat på tre olika sätt man kan jämföra koldioxidutsläpp mellan länder: totala utsläpp, kumulativa utsläpp och per-capita utsläpp. 

Samtliga uppgifter har handlat om länders koldioxidutsläpp, men det har visats på olika sätt. 

---

Nu kör vi ett quiz! Kör kodrutan nedanför!




In [ ]:
#@title Kör för att öppna quizet!
!wget https://raw.githubusercontent.com/lunduniversity/schoolprog-satellite/master/for_developers/quiz.py --quiet
!wget https://raw.githubusercontent.com/lunduniversity/schoolprog-satellite/master/exercises/gapminder/quiz_gapminder.json --quiet
import json
import quiz

with open("quiz_gapminder.json") as f:
  quiz_list = json.load(f)
quiz.main(quiz_list)

---

## Diskussion: Klimaträttvisa

Med klimaträttvisa menas ofta att de länder som har orsakat klimatkrisen ska ta störst ansvar för den. 

Du har nu tittat på tre olika sätt som visar olika länders koldioxidutsläpp. Vilka länder ska då ta störst ansvar för klimatkrisen? Varför?